 # Build CNN from scratch using tensorflow and use on MNIST
 Introducing:
 - tf.stack()
 - tf.get_collection()    &nbsp; &nbsp; &nbsp; &nbsp;  # list down all variables that exist in current graph
 - tf.layers.flatten()
 - tf.reshape()
 - tf.transpose()

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Import MNIST data from tensflow. Already downloaded and stored in MNIST_data folder
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
mnist.train.num_examples

55000

In [4]:
mnist.train.images[1].shape

(784,)

In [13]:
tf.reset_default_graph()

# Build CNN graph

In [14]:
image_batch = tf.placeholder(tf.float32, shape=[None,784], name="images_mini_batch")

In [15]:
image_batch_reshaped = tf.reshape(image_batch, shape=[-1, 28, 28])
image_batch_reshaped.shape

TensorShape([Dimension(None), Dimension(28), Dimension(28)])

In [16]:
# Parameters
s = 3
L1_max_horizontal_iter = int((28-3)/s + 1)
L2_max_horizontal_iter = int((L1_max_horizontal_iter-3)/s + 1)

L1_max_horizontal_iter, L2_max_horizontal_iter

(9, 3)

In [17]:
kernals_L1 = tf.Variable(np.random.rand(3,3,5), dtype=tf.float32, name="layer1_kernals")      # 5 filters (3x3) in layer1
kernals_L2 = tf.Variable(np.random.rand(3,3,5, 10), dtype=tf.float32, name="layer2_kernals")  # 10 filters (3x3x5) in layer2

In [18]:
B1 = tf.Variable(np.random.rand(5), name="L1_biases")    # 5 biases, one each for filters in layer1
B2 = tf.Variable(np.random.rand(10), name="L2_biases")   # 10 biases, one each for filters in layer2

## Build DAG

### CNN Layer1

In [19]:
# Activation Map: Layer1. Activation map is NOT VARIABLE. It's just club of tensorflow ops.

L1_act_list = []
for kernal_idx in range(5):     # Iterate on each filter from layer1
    kernal = kernals_L1[:,:,kernal_idx]
    
    v_op_list = []
    for v_stride in range(L1_max_horizontal_iter):
        h_op_list = []
        for h_stride in range(L1_max_horizontal_iter):
            
            image_patch = image_batch_reshaped[:, v_stride*s:(v_stride+1)*s, h_stride*s:(h_stride+1)*s ]   # Full mini batch is patched
            
            h_op_list.append(tf.reduce_sum(kernal * image_patch, axis=[1,2]))     # Broadcasting used. (reduce_sum is imp)
        
        v_op_list.append(h_op_list)     # Finally one full sheet for each image is created. Depth = number of images
    
    one_kernal_act = tf.stack(v_op_list, name="stacked_one_kernal")    # Just covert to tensor 
    L1_act_list.append(one_kernal_act)

L1_act_map = tf.stack(L1_act_list, axis=3, name = "L1_act_map")  # All activation maps corresponding to every kernal standing parallel to each other.

In [20]:
L1_act_map    # Every of 5 kernals has got its own pillar now. Next layer kernals will have one sub-layer for each pillar

<tf.Tensor 'L1_act_map:0' shape=(9, 9, ?, 5) dtype=float32>

In [21]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)     # Assuring that Activation map is not Variable

[<tf.Variable 'layer1_kernals:0' shape=(3, 3, 5) dtype=float32_ref>,
 <tf.Variable 'layer2_kernals:0' shape=(3, 3, 5, 10) dtype=float32_ref>,
 <tf.Variable 'L1_biases:0' shape=(5,) dtype=float64_ref>,
 <tf.Variable 'L2_biases:0' shape=(10,) dtype=float64_ref>]

In [22]:
# Activation Map: Layer2. Activation map is NOT VARIABLE. It's just club of tensorflow ops.

L2_act_list = []
for kernal_idx in range(10):     # Iterate on each filter from layer2
    kernal = kernals_L2[:,:,:, kernal_idx]           # Single filer of size 3x3x5 is picked
    kernal = tf.reshape(kernal, shape=[3,3,1,5], name="kernal_reshaped")   #3x3x1x5.  One Sub-layer for every pillar
    
    v_op_list = []
    for v_stride in range(L2_max_horizontal_iter):
        h_op_list = []
        for h_stride in range(L2_max_horizontal_iter):
            
            L1_patch = L1_act_map[v_stride*s:(v_stride+1)*s, h_stride*s:(h_stride+1)*s, :, : ]   # One Full mini batch. 3x3x?x5
            
            h_op_list.append(tf.reduce_sum(kernal * L1_patch, axis=[0,1,3]))     # Broadcasting used. (reduce_sum is imp)
        
        v_op_list.append(h_op_list)     
    
    one_kernal_act = tf.stack(v_op_list, name="stacked_one_kernal")    # Just covert to tensor 3x3x?
    L2_act_list.append(one_kernal_act)

L2_act_map = tf.stack(L2_act_list, axis=3, name = "L2_act_map")

In [23]:
L2_act_map.shape

TensorShape([Dimension(3), Dimension(3), Dimension(None), Dimension(10)])

In [24]:
L2_act_map = tf.transpose(L2_act_map, perm=[2,0,1,3])   # Bring the batch dimention to the front, to align it to original input format
L2_act_map.shape

TensorShape([Dimension(None), Dimension(3), Dimension(3), Dimension(10)])

In [25]:
# Flatten the final activation map
L2_flattend = tf.layers.flatten(L2_act_map)        # O/P shape is [batch_size, k]
L2_flattend.shape

TensorShape([Dimension(None), Dimension(90)])

# FC or DNN

# Graph for DNN (Just One layer with 10 neurons) 
**90 L2_flatten outputs feed into it**

In [26]:
Y_OHE_true = tf.placeholder(tf.float32, [None,10], name = "Y_OHE_true")   # Y_true for the mini batch

In [27]:
W = tf.Variable(np.random.rand(90,10), dtype=tf.float32, name="W")
B = tf.Variable(np.random.rand(10), dtype=tf.float32, name="B")

In [28]:
# DAG
Z = tf.add(tf.matmul(L2_flattend,W),B, name="Z")   # Mini-batch output. 10 outputs (10 digits) for each image

In [29]:
#loss
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z, labels=Y_OHE_true, name="loss")  # losses of all images. for mini-batch

In [30]:
loss.shape

TensorShape([Dimension(None)])

In [31]:
cost = tf.reduce_mean(loss)

In [32]:
# Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
set_to_optimize = optimizer.minimize(cost,name="To_Be_Optimized")

In [33]:
summ = tf.summary.scalar("cost", cost)

In [34]:
writer = tf.summary.FileWriter(logdir="./graph", graph=tf.get_default_graph())

In [35]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [41]:
batch_size=1000

for i in range(1000):    # Train iterations
    batch_x , batch_y = mnist.train.next_batch(batch_size)   # Leveraging inbuilt mini-batch generator
    sess.run(set_to_optimize, feed_dict={image_batch:batch_x, Y_OHE_true:batch_y})
    s_buffer = sess.run(summ, feed_dict={image_batch:batch_x, Y_OHE_true:batch_y})    # cost
    writer.add_summary(s_buffer, i)

In [42]:
sess.run(cost, feed_dict={image_batch:batch_x, Y_OHE_true:batch_y})     # Cost at last mini-batch

0.5485609

In [43]:
predictions = sess.run(tf.argmax(Z, axis=1), feed_dict={image_batch:mnist.test.images})

acc = (predictions == np.argmax(mnist.test.labels,axis=1))
acc

array([ True,  True,  True, ...,  True, False,  True])

In [44]:
acc.mean()

0.8502

In [45]:
acc.sum()

8502

In [46]:
sess.close()
writer.close()